In [ ]:
import multiprocessing
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import StarmapParallelization
from pymoo.constraints.as_obj import ConstraintsAsObjective
import optimize as op



class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


problem = MyProblem()

opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 0.6
opt.getLiftoffState()
print(opt.velocityConstraint(opt.bp_init_guess))
print(opt.multiObjective(opt.bp_init_guess))


class CorgiProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=24,
                         n_obj=1,
                         n_ieq_constr=24,
                         xl=np.array([0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05]),
                         xu=np.array([0.05, 0.02, 1, 0.5, 1, 0.5,
                                      0.05, 0.02, 1, 0.5, 1, 0.5,
                                      0.05, 0.02, 1, 0.5, 1, 0.5,
                                      0.05, 0.02, 1, 0.5, 1, 0.5]))

    def _evaluate(self, x, out, *args, **kwargs):
        # cost_s, cost_u = opt.multiObjective(x)
        # f1 = cost_s
        # f2 = cost_u
        f  = opt.objective(x)

        g1 = -1 * opt.innerWsPFConstraint(x)
        g2 = -1 * opt.outerWsPFConstraint(x)
        g3 = np.abs(opt.velocityConstraint(x)) - 0.01
        g = np.hstack((g1, g2))
        g = np.hstack((g, g3))

        out["F"] = [f]
        out["G"] = list(g)


problem = CorgiProblem()


In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

algorithm = NSGA2(
    pop_size=40,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
from pymoo.termination import get_termination

termination = get_termination("n_gen", 100)

In [54]:
from pymoo.optimize import minimize

result = minimize(ConstraintsAsObjective(problem),
                  algorithm,
                  termination,
                  seed=1,
                  save_history=True,
                  verbose=True)


iter 25 cost 23.91523041850437
[0.0354824  0.0112106  0.67501249 0.1438363  0.85727008 0.05727062]
[0.04494642 0.00579205 0.77606129 0.18525553 0.80962143 0.14444417]
[0.03427386 0.01534569 0.31555553 0.35328479 0.74702842 0.07176819]
[0.03428224 0.00397896 0.15109315 0.15739543 0.03636161 0.0110062 ]
iter 26 cost 18.41909495655792
[0.04349128 0.01599553 0.08972423 0.07743627 0.21554941 0.45453346]
[0.04422629 0.01107769 0.31147281 0.40916973 0.4409529  0.45790092]
[0.04412104 0.00956063 0.134948   0.48682395 0.16823531 0.06143117]
[0.03862364 0.00808404 0.15528397 0.35112541 0.19685762 0.3041426 ]
iter 27 cost 2.3591383675372644
[0.04508612 0.00421465 0.60494471 0.36191061 0.64183652 0.27842002]
[ 0.03590965  0.01463213  0.94585536  0.18405876  0.78436    -0.01912243]
[0.04670543 0.003845   0.4011459  0.11504458 0.08930261 0.44754705]
[ 0.03740308  0.01061395  0.4991751   0.02268863  0.21438952 -0.00809616]
iter 28 cost 9.091200016158897
[0.04015843 0.00523099 0.36349099 0.00943593 0.

In [55]:
print(repr(result.X))
print(result.F)
print(result.X[-1])

array([[ 3.18378372e-02,  3.55046204e-03,  1.59935284e-02,
         9.19795465e-02,  1.61942036e-02, -4.36550055e-02,
         3.09768098e-02,  1.68010109e-03,  1.70345884e-01,
         4.85240172e-02,  1.81190990e-01,  8.10412482e-02,
         3.10693601e-02,  4.81596482e-03,  1.42909379e-02,
         3.02639358e-01,  1.02002438e-02, -1.05290266e-02,
         3.02139686e-02,  1.04700952e-03,  2.28231662e-02,
        -4.48515435e-02,  8.38525397e-02,  3.25158566e-02],
       [ 3.18399561e-02,  2.37126925e-03,  1.59976037e-02,
         9.12061053e-02,  1.31900820e-02, -4.36550313e-02,
         3.13821637e-02,  1.05475752e-03,  1.48568272e-02,
         5.22351404e-02,  1.70689460e-01, -2.48591109e-02,
         3.10803506e-02,  4.05238052e-03,  1.53632034e-02,
         3.01436676e-01,  2.07889140e-02, -2.59166132e-02,
         3.01660188e-02,  2.24766073e-03,  1.08158983e-02,
        -4.56564738e-02,  2.19271266e-01,  3.35407808e-02],
       [ 3.20601157e-02,  2.33573973e-03,  1.01097882e

In [56]:
# print(result.pop.get("X"))
bp = np.array([0.034815,    0.00341456,  0.01004385, -0.03486108,  0.02724615, -0.08330442,
 0.03140377,  0.00436724,  0.01425195, -0.03556327,  0.02642443, -0.09868557,
 0.03100124,  0.0039261,   0.02582531, -0.02476453,  0.01291202, -0.04367011,
 0.03027373,  0.00167466,  0.0149596,  -0.03968913,  0.02813969, -0.09163613])

bp = np.array([ 3.90475000e-02,  1.26541517e-02,  1.29990659e-02,
        -9.43200192e-02,  5.01530731e-02, -8.84345508e-02,
         3.14253998e-02,  6.60885379e-03,  1.64677686e-01,
        -7.29851060e-02,  4.03009612e-02, -5.70694902e-02,
         3.10178224e-02,  1.41883854e-02,  2.16441144e-02,
         3.29020311e-02,  1.51452422e-02, -9.34873236e-02,
         3.01523820e-02,  1.95010099e-03,  4.55182621e-02,
        -9.75865250e-02,  2.84592035e-02, -5.60758920e-02])
# opt.exportBezierData(bp)
# opt.exportBezierData(result.X[-5])